In [16]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
from scipy.cluster import hierarchy as sch
import pandas as pd
import numpy as np

import yfinance as yf

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import (BUSINESS_DAYS_IN_YEAR,
                                   standardDeviation)
from _utils.portfoliohandcrafiting import (
    create_fx_series_given_adjusted_prices_dict,
    calculate_variable_standard_deviation_for_risk_targeting_from_dict,
    calculate_position_series_given_variable_risk_for_dict,
    calculate_perc_returns_for_dict,
    calculate_perc_returns_for_dict_with_costs,
)
from _utils.strategies.trend_simple_filter import (
    apply_buffering_to_position_dict
)
from _utils.strategies.simple_carry import *
from _utils.normalized_price import calculate_normalised_price_dict


DATA_DIR = "../_databases"

DEFAULT_DATE_FORMAT = "%Y-%m-%d"


In [17]:
#%% GET DATA
# # =================

def get_data_dict(instrument_list: list):

    all_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]
    )

    adjusted_prices = dict(
        [
            (instrument_code, data_for_instrument.adjusted)
            for instrument_code, data_for_instrument in all_data.items()
        ]   
    )

    current_prices = dict(
        [
            (instrument_code, data_for_instrument.underlying)
            for instrument_code, data_for_instrument in all_data.items()
        ]
    )
    
    carry_data = dict(
        [
            (instrument_code, pd.read_csv((DATA_DIR + '/' + f'{instrument_code}' + '_carry.csv'),index_col='index'))
            for instrument_code in instrument_list
        ]   
    )

    return adjusted_prices, current_prices, carry_data

In [18]:
adjusted_prices_dict, current_prices_dict, carry_prices_dict = get_data_dict(['sp500','gas'])
multipliers = dict(sp500=5, gas=10000)
risk_target_tau = 0.2

fx_series_dict = create_fx_series_given_adjusted_prices_dict(adjusted_prices_dict,dict(sp500=1, gas=1))

capital = 2000000
idm = 1.5
instrument_weights = dict(sp500=0.5, gas=0.5)
cost_per_contract_dict = dict(sp500=0.875, gas=15.3)

std_dev_dict = calculate_variable_standard_deviation_for_risk_targeting_from_dict(
adjusted_prices=adjusted_prices_dict,
current_prices=current_prices_dict,
annualise_stdev=True,  ## can also be False if want to use daily price diff
use_perc_returns=True,  ## can also be False if want to use daily price diff
)

In [19]:
average_position_contracts_dict = (
        calculate_position_series_given_variable_risk_for_dict(
            capital=capital,
            risk_target_tau=risk_target_tau,
            idm=idm,
            weights=instrument_weights,
            std_dev_dict=std_dev_dict,
            fx_series_dict=fx_series_dict,
            multipliers=multipliers,
        )
    )

normalised_price_dict = calculate_normalised_price_dict(
       adjusted_prices_dict=adjusted_prices_dict,
       std_dev_dict=std_dev_dict,
   )

rule_trend_carry_spec = [
        dict(function="carry", span=5),
        dict(function="carry", span=20),
        dict(function="carry", span=60),
        dict(function="carry", span=120),
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]


position_contracts_normalized_trend_dict = calculate_position_dict_with_forecast_applied(
        adjusted_prices_dict=normalised_price_dict,
        carry_prices_dict=carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=average_position_contracts_dict,
        rule_spec=rule_trend_carry_spec,
    )


buffered_position_normalized_trend_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_normalized_trend_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_normalized_trend_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_normalized_trend_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)




port_normalized_trend_return = aggregate_returns(
    perc_return_normalized_trend_dict)

port_normalized_trend_return.sort_index(inplace=True)

# ===============

position_contracts_carry_trend_dict = calculate_position_dict_with_forecast_applied(
        adjusted_prices_dict=
        adjusted_prices_dict,
        carry_prices_dict=
        carry_prices_dict,
        std_dev_dict=std_dev_dict,
        average_position_contracts_dict=
        average_position_contracts_dict,
        rule_spec=rule_trend_carry_spec,
    )


buffered_position_carry_trend_dict = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_carry_trend_dict,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_carry_trend_dict = calculate_perc_returns_for_dict(
    position_contracts_dict=buffered_position_carry_trend_dict,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
)

port_carry_trend_return = aggregate_returns(
    perc_return_carry_trend_dict)

port_carry_trend_return.sort_index(inplace=True)

#=================================

rules_spec_ewmac = [
        dict(function="ewmac", fast_span=16),
        dict(function="ewmac", fast_span=32),
        dict(function="ewmac", fast_span=64),
    ]

position_contracts_dict_ewmac = calculate_position_dict_with_forecast_applied(
    adjusted_prices_dict=adjusted_prices_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    rule_spec=rules_spec_ewmac,
)

buffered_position_dict_ewmac = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_dict_ewmac,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_dict_ewmac = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_dict_ewmac,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)

perc_return_aggregated_ewmac = aggregate_returns(perc_return_dict_ewmac)

rules_spec_carry = [
    dict(function="carry", span=5),
    dict(function="carry", span=20),
    dict(function="carry", span=60),
    dict(function="carry", span=120),
]
position_contracts_dict_carry = calculate_position_dict_with_forecast_applied(
    adjusted_prices_dict=adjusted_prices_dict,
    carry_prices_dict=carry_prices_dict,
    std_dev_dict=std_dev_dict,
    average_position_contracts_dict=average_position_contracts_dict,
    rule_spec=rules_spec_carry,
)

buffered_position_dict_carry = apply_buffering_to_position_dict(
    position_contracts_dict=position_contracts_dict_carry,
    average_position_contracts_dict=average_position_contracts_dict,
)

perc_return_dict_carry = calculate_perc_returns_for_dict_with_costs(
    position_contracts_dict=buffered_position_dict_carry,
    fx_series=fx_series_dict,
    multipliers=multipliers,
    capital=capital,
    adjusted_prices=adjusted_prices_dict,
    cost_per_contract_dict=cost_per_contract_dict,
    std_dev_dict=std_dev_dict,
)

perc_return_aggregated_carry = aggregate_returns(perc_return_dict_carry)

starting_portfolio = (
    perc_return_aggregated_ewmac * 0.6 + perc_return_aggregated_carry * 0.4
)

relative_performance = perc_return_aggregated_ewmac - perc_return_aggregated_carry
rolling_12_month = relative_performance.rolling(BUSINESS_DAYS_IN_YEAR).mean()

relative_performance = perc_return_aggregated_ewmac - perc_return_aggregated_carry
rolling_12_month = (
    relative_performance.rolling(BUSINESS_DAYS_IN_YEAR).sum() / risk_target_tau
)

    # W t = EWMA span=30 (min(1, max(0, 0.5 + RP t ÷ 2)))
raw_weighting = 0.5 + rolling_12_month / 2
clipped_weighting = raw_weighting.clip(lower=0, upper=1)
smoothed_weighting = clipped_weighting.ewm(30).mean()

weighted_portfolio = (
    perc_return_aggregated_ewmac * 0.5 * smoothed_weighting
    + perc_return_aggregated_carry * 0.5 * (1 - smoothed_weighting)
)




In [20]:
# --- Plot retorno acumulado
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=port_normalized_trend_return.index,
        y=port_normalized_trend_return.cumsum()*100,
        name="Normalized Trend",
        line=dict(color="green"),
    )
)

fig.add_trace(
    go.Scatter(
        x=port_carry_trend_return.index,
        y=port_carry_trend_return.cumsum()*100,
        name="Carry & Trend",
        line=dict(color="red"),
    )
)

fig.add_trace(
    go.Scatter(
        x=weighted_portfolio.index,
        y=weighted_portfolio.cumsum()*100,
        name="Carry & Trend with relative performance",
        line=dict(color="blue"),
    )
)



fig.update_layout(
    title="50% SP500 Fut + 50% GAS Fut Portfolio",
    xaxis_title="Data",
    yaxis_title="Acumulated return (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=0.8,
        xanchor="left",
        x=0),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho")
print("Estatísticas Portfolio Normalized and Trend Frequência Anual")
print(calculate_stats(port_normalized_trend_return, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Carry and Trend with Relative Alocation Frequência Anual")
print(calculate_stats(weighted_portfolio, freq=YEAR))
print("================================")
print("Estatísticas Portfolio Carry and Trend Frequência Anual")
print(calculate_stats(port_carry_trend_return, freq=YEAR))



Estatísticas de Desempenho
Estatísticas Portfolio Normalized and Trend Frequência Anual
{'ann_mean': 25.344055143654536, 'ann_std': 3.451784017087763, 'sharpe': 7.342306186653321, 'skew': -0.6042718552716223, 'avg_drawdown': 0.07788144408181816, 'max_drawdown': 0.4713746437499997, 'quant_lower': 1.3713974138001805, 'quant_upper': 0.5348642015000258}
Estatísticas Portfolio Carry and Trend with Relative Alocation Frequência Anual
{'ann_mean': 12.426035784384183, 'ann_std': 1.858824404155032, 'sharpe': 6.684889522973904, 'skew': 0.3801118368882601, 'avg_drawdown': 0.04100089303036126, 'max_drawdown': 0.1807927157629443, 'quant_lower': 1.5791649873978133, 'quant_upper': 1.4699314334886446}
Estatísticas Portfolio Carry and Trend Frequência Anual
{'ann_mean': 17.357224749327266, 'ann_std': 3.218435521405167, 'sharpe': 5.393062757941819, 'skew': -0.0719458730678966, 'avg_drawdown': 0.11416057889696724, 'max_drawdown': 0.5409746924999894, 'quant_lower': 0.5175038583970281, 'quant_upper': 0.746